In [39]:
import librosa

In [40]:
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [41]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype = "float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

In [42]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=20)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)                             
    return mfccs_scaled_features

In [43]:
# def load_data(test_size=0.2):
#     x,y = [],[]
#     for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
#         label = os.path.basename(file)
#         for audio in glob.glob(file+"\*.wav"):
# #             print(audio)
#             feature = featureExtractor(audio)
#             x.append(feature)
#             y.append(label)
#     return train_test_split(np.array(x), np.array(y), test_size=test_size, random_state=101)

In [44]:
# x_train, x_test, y_train, y_test=load_data()

In [45]:
# print(len(x_train),len(y_train),len(x_test),len(y_test))
# print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [46]:
extracted_features=[]
def load_data():
    c=0
    for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
        label = os.path.basename(file)
        for audio in glob.glob(file+"\*.wav"):
#             print(audio)
            if(label=='hungry'):
                if(c>200):
                    continue
                c+=1
            feature = featureExtractor(audio)
            extracted_features.append([feature,label])

In [47]:
load_data()

In [48]:
# extracted_features

In [49]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-497.25443, 110.52981, -51.0884, -15.25888, 1...",belly_pain
1,"[-497.26553, 151.22801, -56.617985, -1.158033,...",belly_pain
2,"[-421.2681, 153.3781, -55.931305, -7.050015, 6...",belly_pain
3,"[-383.0191, 164.00575, -70.79439, -8.87635, 8....",belly_pain
4,"[-310.78342, 171.10025, -66.29349, 4.647664, 1...",belly_pain


In [50]:
print(extracted_features_df.shape)

(276, 2)


In [51]:
# audio_file_path='belly_pain.wav'
# mfcc=featureExtractor(audio_file_path)
# print(mfcc)
# print(mfcc.shape)

In [52]:
X=np.array(extracted_features_df['feature'].tolist())
Y=np.array(extracted_features_df['class'].tolist())

In [53]:
print(X.shape,Y.shape)
print(Y)

(276, 20) (276,)
['belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain' 'belly_pain'
 'belly_pain' 'burping' 'burping' 'burping' 'burping' 'burping' 'burping'
 'burping' 'burping' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'discomfort' 'discomfort'
 'discomfort' 'discomfort' 'discomfort' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry' 'hungry'
 'hungry' 'hung

In [54]:
Y.shape

(276,)

In [55]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(Y))
# Y=np.array(pd.get_dummies(Y))

In [56]:
print(y.shape)
y

(276, 5)


array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [57]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [58]:
X_train.shape

(220, 20)

In [59]:
y_train.shape

(220, 5)

In [60]:
X_test.shape

(56, 20)

In [61]:
y_test.shape

(56, 5)

In [62]:
# BUILDING MODEL
import tensorflow as tf
print(tf.__version__)

2.4.1


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [64]:
num_labels=y.shape[1]

In [65]:
# Dense()

In [66]:
# ANN
model=Sequential()
#first layer
model.add(Dense(100,input_shape=(20,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
          
# final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [67]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               2100      
_________________________________________________________________
activation_4 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_5 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                

In [68]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [69]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs=50
num_batch_size=32

checkpointer=ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                             verbose=1,save_best_only=True)
start=datetime.now()
model.fit(X_train,y_train,batch_size=num_batch_size,epochs=num_epochs,
          validation_data=(X_test,y_test),callbacks=[checkpointer] )
duration=datetime.now() - start
print("Training completed in time: ",duration)

Epoch 1/50
7/7 [==============================] - 3s 259ms/step - loss: 41.6149 - accuracy: 0.3871 - val_loss: 14.9847 - val_accuracy: 0.7857

Epoch 00001: val_loss improved from inf to 14.98472, saving model to saved_models\audio_classification.hdf5
Epoch 2/50
7/7 [==============================] - 0s 26ms/step - loss: 32.6679 - accuracy: 0.5508 - val_loss: 13.6108 - val_accuracy: 0.7857

Epoch 00002: val_loss improved from 14.98472 to 13.61083, saving model to saved_models\audio_classification.hdf5
Epoch 3/50
7/7 [==============================] - 0s 28ms/step - loss: 28.5419 - accuracy: 0.5483 - val_loss: 7.8117 - val_accuracy: 0.7857

Epoch 00003: val_loss improved from 13.61083 to 7.81166, saving model to saved_models\audio_classification.hdf5
Epoch 4/50
7/7 [==============================] - 0s 31ms/step - loss: 23.8673 - accuracy: 0.4877 - val_loss: 7.5440 - val_accuracy: 0.7857

Epoch 00004: val_loss improved from 7.81166 to 7.54399, saving model to saved_models\audio_classific

In [70]:
loss,test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print("loss: ",loss)
print(test_accuracy)

loss:  1.1338931322097778
0.75


In [71]:
filename='burping.wav'
prediction_feature=featureExtractor(filename).reshape(1,-1)
# model.predict_classes(prediction_feature)
predicted_label=model.predict_classes(prediction_feature)
print(predicted_label)
predcited_class=labelencoder.inverse_transform(predicted_label)
predcited_class
# (model.predict(prediction_feature) > 0.5).astype("int32")

[3]


C:\Users\Rajeen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array(['hungry'], dtype='<U10')

In [72]:
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

In [73]:
from sklearn.metrics import classification_report,confusion_matrix,f1_score
print(classification_report(y_true=label,y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         3
           3       0.80      0.93      0.86        44
           4       0.20      0.25      0.22         4

    accuracy                           0.75        56
   macro avg       0.20      0.24      0.22        56
weighted avg       0.65      0.75      0.69        56



C:\Users\Rajeen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
f1_score_per_class_validation = f1_score(y_true=label,y_pred=y_pred,average=None) 
print("The f1-score per class: ",f1_score_per_class_validation)
print("The f1-score : ",f1_score(y_true=label,y_pred=y_pred,average='weighted'))

The f1-score per class:  [0.         0.         0.         0.86315789 0.22222222]
The f1-score :  0.6940685045948204
